プログラムの構成
#1.事前設定:oandapyのインストール, googlecolabのドライブマウント, setup_fxconn()
#2.為替相場から値を取得:fetch_fxvalue()
#3.データ成形し、特徴量抽出:calc_feat()
#4.モデル学習:build_model()
#5.将来の為替変動予測:predict()
#6.予測結果をもとに取引の戦略を立てる:build_strategy()
#7.取引:transaction_with_strategy()

In [54]:
#1.事前設定
#oandapyインストール
! pip install git+https://github.com/oanda/oandapy.git

  Cloning https://github.com/oanda/oandapy.git to c:\users\satoshi\appdata\local\temp\pip-req-build-kx_ol5_e
  Running setup.py bdist_wheel for oandapy: started
  Running setup.py bdist_wheel for oandapy: finished with status 'done'
  Stored in directory: C:\Users\satoshi\AppData\Local\Temp\pip-ephem-wheel-cache-cerlsp6k\wheels\84\33\ea\2d5dddc641a73a23c531dd327e6e743ccbad487a2ae6f38c30
Successfully built oandapy


You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
#google collaboration上で、Google ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [55]:
#2. 為替相場から値を取得:fetch_fxvalue()
# API接続設定のファイルを読み込む
import configparser
import oandapy
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pytz
import datetime
from datetime import datetime, timedelta


#TODO: 外部から読み込む形にしたい
#path_oanda='./drive/My Drive/oandapy' #google colac用
path_oanda='..'
path_config=path_oanda+'/data/config.txt'

config = configparser.ConfigParser()
config.read(path_config)

#TODO: グローバル変数にしたくない
instrument = config['oanda']['instrument']
granularity = config['oanda']['granularity']
count = int(config['oanda']['count'])
api_key = config['oanda']['api_key']
account_id = config['oanda']['account_id']


def create_api_fxconn(path_config):
# API接続
    oanda = oandapy.API(environment="practice", access_token=api_key)
    return oanda


def iso_jp(iso):
#日付の調整する
#文字列 -> datetime
     date = None
     try:
         date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
         date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
     except ValueError:
         try:
             date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
             date = dt.astimezone(pytz.timezone("Asia/Tokyo"))
         except ValueError:
             pass
     return date

  
def create_df_fxvalue_latesthistory_rawdata(oanda):
#生データのdfを生成
    instrument, granularity, count = (config['oanda']['instrument'], 
                                      config['oanda']['granularity'], 
                                      int(config['oanda']['count']))
    json_response = oanda.get_history(instrument=instrument, 
                                      granularity=granularity, 
                                      count=count)
    df_response = pd.DataFrame(json_response["candles"])
    df_response["time"] = df_response["time"].apply(lambda x: iso_jp(x))
    return df_response

#-----------------main-----------------------------

def fetch_fxvalue(path_config):
#2のメイン関数, 為替の値を取得
    instrument, granularity, count = (config['oanda']['instrument'], 
                                      config['oanda']['granularity'], 
                                      int(config['oanda']['count']))
    oanda = create_api_fxconn(path_config=path_config)
    df_fxvalue_latesthistory_rawdata = create_df_fxvalue_latesthistory_rawdata(oanda)
    return df_fxvalue_latesthistory_rawdata 
    

In [56]:
#2 テスト
path_oanda='..'
path_config=path_oanda+'/data/config.txt'
tmp = fetch_fxvalue(path_config)
tmp.head()

,closeAsk,closeBid,complete,highAsk,highBid,lowAsk,lowBid,openAsk,openBid,time,volume
0,109.808,109.795,True,109.823,109.812,109.808,109.795,109.820,109.808,2020-02-06 00:07:00+09:00,77
1,109.794,109.782,True,109.809,109.796,109.788,109.775,109.809,109.796,2020-02-06 00:08:00+09:00,128
2,109.808,109.796,True,109.808,109.796,109.794,109.782,109.795,109.783,2020-02-06 00:09:00+09:00,78
3,109.826,109.814,True,109.827,109.815,109.797,109.783,109.810,109.797,2020-02-06 00:10:00+09:00,118
4,109.824,109.813,True,109.839,109.828,109.822,109.809,109.828,109.815,2020-02-06 00:11:00+09:00,81


In [49]:
#3. 特徴量抽出:feat_calc()
# API接続設定のファイルを読み込む

def data_maker(data, window_len):
    data_lstm_in=[]
    if len(data)==window_len:
        temp = data[:window_len].copy()
        temp = temp / temp.iloc[0] - 1
        data_lstm_in.append(temp)
    for i in range(len(data) - window_len):
        temp = data[i:(i + window_len)].copy()
        temp = temp / temp.iloc[0] - 1
        data_lstm_in.append(temp)
    return data_lstm_in


def pd_to_np(data_lstm_in):
#うんこみたいな関数すぎる、変えたい
#もともとdf？listになってるんやけど
    data_lstm_in = [np.array(data_lstm_input) for data_lstm_input in data_lstm_in]  #array のリスト
    data_lstm_in = np.array(data_lstm_in) #np.array
    return data_lstm_in


def extract_feature(df_rawdata, ratio_num_traindata):
#特徴量抽出->ここをいじります
#ratio_num_traindataは訓練データと検証用データの分割比
#TODO: ratio_num_traindataは引数で与えたい

    #1. 特徴量算出に必要なカラムを設定
    df = df_rawdata[['time', 'openAsk']]
    df.columns = ['time', 'open']

    #2. 予測に使うデータ数を設定
    window_len = 10
    #3. 訓練データと検証用データを所定の割合で分割
    #TODO: ごちゃごちゃしているので、書き換えたい
    train, test = df[df.index < int(len(df)*ratio_num_traindata)], \
                    df[df.index >= int(len(df)*ratio_num_traindata)]
    latest = test[:window_len]
    del train['time']
    del test['time']
    del latest['time']
    length = len(test)- window_len
    
    #モデル入力、モデル出力用にデータ成形
    ##ここを直して、特徴量抽出のアルゴリズム考える
    #TODO: data_makerとかpd_to_npとかいう意味わからん関数書き換えたい
    #TODO: latestがよくわからん
    train_lstm_in = data_maker(train, window_len)
    lstm_train_out = (train['open'][window_len:].values / train['open'][:-window_len].values)-1
    test_lstm_in = data_maker(test, window_len)
    lstm_test_out = (test['open'][window_len:].values / test['open'][:-window_len].values)-1
    latest_lstm_in = data_maker(latest, window_len)
    train_lstm_in = pd_to_np(train_lstm_in)
    test_lstm_in = pd_to_np(test_lstm_in)
    latest_lstm_in = pd_to_np(latest_lstm_in)
    
    #入出力の名前とかが意味わからなかったので、dictで管理してる
    X_dict = {'train':train_lstm_in, 'test':test_lstm_in, 'latest':latest_lstm_in}
    y_dict = {'train':lstm_train_out, 'test':lstm_test_out}

    return X_dict, y_dict

In [50]:
#3テスト
X_dict, y_dict = extract_feature(tmp, ratio_num_traindata=0.8)

In [52]:
#4 モデル学習: train_model()

#TODO importはutil.pyにまとめる
from keras.models import Sequential
from keras.layers import Activation, Dense,Bidirectional
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

np.random.seed(202)


#モデルのハイパーパラメータ設定
#TODO もっとすっきり書きたい→csvとかに書いておいて、pd.read_csv()とするといい？
def set_model_param_dict():
    param_dict = dict()
    param_dict['activation']="relu"
    param_dict['dropout']=0.25
    param_dict['loss']="mae"
    param_dict['optimizer']="adam"
    param_dict['num_LSTMcells']=248
    param_dict['recurrent_dropout']=0.3

    param_dict['num_units_Dense1']=64
    param_dict['num_units_Dense2']=32

    param_dict['epochs']=10
    param_dict['batch_size']=1
    param_dict['flag_shuffle_traindata']=True

    return param_dict

#出力データのサイズ抽出
def extract_output_size(output_arr):
    if output_arr.ndim == 1:
      output_size = 1
    else: 
      output_size = output_arr.shape[1]
    
    return output_size

#===========================================================
#モデルの定義
def define_model(X, y, param_dict):
    #TODO 変数名と同じ要素を取ってくるようにしたい：assign_param(model_params, given_params)
    #クラスとかをつかうとよい？
    #最も理想param_list=param_dict**からのnum_cellとかに値代入されている
    #ネットワーク構造を決定するパラメタ群は別の変数にしたい
    activation = param_dict['activation']
    dropout = param_dict['dropout']
    loss = param_dict['loss']
    optimizer = param_dict['optimizer']
    num_LSTMcells = param_dict['num_LSTMcells']
    recurrent_dropout = param_dict['recurrent_dropout']
    
    #shape[0]は特徴量ベクトルの数
    #shape[1]は時間方向のデータ数
    #shape[2]は他のなにか
    input_shape = (X.shape[1], X.shape[2])

    num_units_Dense1 = param_dict['num_units_Dense1']
    num_units_Dense2 = param_dict['num_units_Dense2']
    num_dim_output = extract_output_size(y)
    #TODO 上記全部２つの関数にまとめたい：set_hyperparamer, extract_output_size


    #モデルの定義
    model = Sequential()
    model.add(Bidirectional(LSTM(num_LSTMcells, dropout=dropout,recurrent_dropout=recurrent_dropout,input_shape=input_shape)))
    model.add(Dense(num_units_Dense1,activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(num_units_Dense2,activation=activation))
    model.add(Dense(num_dim_output))
 
    model.compile(loss=loss, optimizer=optimizer)
    return model



#モデル学習(一応historyも返すようにしている)
def train_model_with_history(defined_model, X_train, y_train, param_dict):
    epochs = param_dict['epochs']
    batch_size = param_dict['batch_size']
    flag_shuffle_traindata = param_dict['flag_shuffle_traindata'] 
    
    history = defined_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=flag_shuffle_traindata)
    return model, history



#メイン：モデルを定義、学習を行いファイルを生成
def build_model(X_train, y_train, param_dict):
    defined_model = define_model(X=X_train, y=y_train, param_dict=param_dict)
    model, history = train_model_with_history(defined_model=defined_model, X_train=X_train, y_train=y_train, param_dict=param_dict)
    return model #arbitrarily add -> history


W0126 14:28:15.028591 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0126 14:28:16.147869 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0126 14:28:16.441787 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0126 14:28:17.533325 18344 deprecation_wrapper.py:119] From c:\users\satoshi\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_defa

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
#4 テスト
X_train, y_train = X_dict['train'], y_dict['train']
param_dict = set_model_param_dict()
model = build_model(X_train, y_train, param_dict)